In [1]:
# Import the libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Data loading

In [2]:
apartments = pd.read_csv("Data/kursk_apartments_raw.csv",index_col=0)
apartments.head()

,Адрес,Район,Цена,Количество комнат,Общая площадь,Площадь кухни,Этаж,Балкон или лоджия,Тип комнат,Санузел,...,Интернет и ТВ,Можно с детьми,Можно с животными,Можно курить,Максимум гостей,Тип дома,Этажей в доме,Пассажирский лифт,Грузовой лифт,Парковка
0,"Курская область, Курск, ул. Карла Либкнехта, 16А",р-н Центральный,35 000 ₽ в месяц,3,116 м²,20 м²,5 из 9,"балкон, лоджия",изолированные,раздельный,...,"Wi-Fi, телевидение",да,нет,нет,NaN,кирпичный,9,1,нет,за шлагбаумом во дворе
1,"Курская область, Курск, Красная пл., 2/4",р-н Центральный,27 000 ₽ в месяц,3,80 м²,15 м²,2 из 5,балкон,изолированные,совмещенный,...,Wi-Fi,да,нет,нет,NaN,кирпичный,5,NaN,NaN,за шлагбаумом во дворе
2,"Курская область, Курск, пр-т Вячеслава Клыкова...",р-н Центральный,12 000 ₽ в месяц,2,58 м²,12 м²,1 из 17,балкон,изолированные,раздельный,...,телевидение,да,нет,нет,NaN,панельный,17,1,1,открытая во дворе
3,"Курская область, Курск, пр-т Победы, 2",р-н Центральный,13 000 ₽ в месяц,2,75 м²,15 м²,5 из 10,лоджия,изолированные,NaN,...,Wi-Fi,да,нет,нет,NaN,монолитный,10,1,NaN,открытая во дворе
4,"Курская область, Курск, микрорайон Северо-Запа...",р-н Центральный,15 000 ₽ в месяц,3,85 м²,13 м²,9 из 10,лоджия,"изолированные, смежные",раздельный,...,NaN,да,нет,нет,NaN,панельный,10,NaN,NaN,открытая во дворе


In [3]:
apartments.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 217 entries, 0 to 216
Data columns (total 23 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Адрес              217 non-null    object 
 1   Район              215 non-null    object 
 2   Цена               217 non-null    object 
 3   Количество комнат  217 non-null    object 
 4   Общая площадь      217 non-null    object 
 5   Площадь кухни      213 non-null    object 
 6   Этаж               217 non-null    object 
 7   Балкон или лоджия  169 non-null    object 
 8   Тип комнат         108 non-null    object 
 9   Санузел            187 non-null    object 
 10  Ремонт             217 non-null    object 
 11  Мебель             181 non-null    object 
 12  Техника            194 non-null    object 
 13  Интернет и ТВ      107 non-null    object 
 14  Можно с детьми     217 non-null    object 
 15  Можно с животными  217 non-null    object 
 16  Можно курить       183 non

In [4]:
apartments.isnull().sum()

Адрес                  0
Район                  2
Цена                   0
Количество комнат      0
Общая площадь          0
Площадь кухни          4
Этаж                   0
Балкон или лоджия     48
Тип комнат           109
Санузел               30
Ремонт                 0
Мебель                36
Техника               23
Интернет и ТВ        110
Можно с детьми         0
Можно с животными      0
Можно курить          34
Максимум гостей      217
Тип дома               0
Этажей в доме          0
Пассажирский лифт     57
Грузовой лифт         99
Парковка              60
dtype: int64

In [5]:
apartments.dropna(subset=['Район'], axis=0, inplace=True)
apartments.drop('Максимум гостей', axis=1, inplace=True)

# Converting numeric fields to float dtype

In [6]:
def convert_to_float_by_splitter(value, splitter):
    if splitter is None:
        raise ValueError("splitter must be assigned")
    if value is None:
        return None    
    
    try:
        value = str(value).split(splitter)[0].replace(' ','')
        return float(value)
    except ValueError:
        return None

apartments.loc[:, 'Цена'] = apartments['Цена'].apply(lambda x: convert_to_float_by_splitter(x, '₽'))
apartments.loc[:, 'Общая площадь'] = apartments['Общая площадь'].apply(lambda x: convert_to_float_by_splitter(x, 'м'))
apartments.loc[:, 'Площадь кухни'] = apartments['Площадь кухни'].apply(lambda x: convert_to_float_by_splitter(x, 'м'))

In [7]:
apartments['Площадь кухни'].fillna(apartments.groupby('Общая площадь')['Площадь кухни'].transform('mean'), inplace=True)

In [8]:
apartments[apartments['Площадь кухни'].isnull()]

,Адрес,Район,Цена,Количество комнат,Общая площадь,Площадь кухни,Этаж,Балкон или лоджия,Тип комнат,Санузел,...,Техника,Интернет и ТВ,Можно с детьми,Можно с животными,Можно курить,Тип дома,Этажей в доме,Пассажирский лифт,Грузовой лифт,Парковка
109,"Курская область, Курск, ул. Димитрова, 40",р-н Центральный,20000.0,студия,32.3,NaN,4 из 9,лоджия,NaN,раздельный,...,"кондиционер, холодильник, плита, микроволновка...","Wi-Fi, телевидение",да,нет,нет,кирпичный,9,1,нет,за шлагбаумом во дворе
162,"Курская область, Курск, Северный район, ул. Ге...",р-н Центральный,7500.0,студия,29.0,NaN,4 из 5,балкон,NaN,совмещенный,...,"холодильник, плита, стиральная машина",Wi-Fi,нет,нет,нет,монолитно-кирпичный,5,нет,нет,открытая во дворе


In [9]:
kitchen_mean = apartments[apartments['Общая площадь'] == 30].groupby('Общая площадь')['Площадь кухни'].transform('mean')
apartments['Площадь кухни'].fillna(kitchen_mean.values[0], inplace=True)

In [10]:
apartments['Площадь кухни'].isnull().sum()

0

In [11]:
def convert_floor_to_int(floor):
    if floor is None:
        return None
    try:
        floor = str(floor).split('из')[0].strip()
        return int(floor)
    except ValueError:
        return None
    
apartments.loc[:, 'Этаж'] = apartments['Этаж'].apply(convert_floor_to_int)

In [12]:
pd.concat([apartments['Грузовой лифт'], apartments['Пассажирский лифт']], axis=0).unique()

array(['нет', nan, '1', '4', '2'], dtype=object)

In [13]:
def have_elevator(value):
    if value is None:
        return 0
    if str(value).isdigit():
        return 1
    return 0

apartments.loc[:, 'Грузовой лифт'] = apartments['Грузовой лифт'].apply(have_elevator)
apartments.loc[:, 'Пассажирский лифт'] = apartments['Пассажирский лифт'].apply(have_elevator)
apartments['Лифт'] = (apartments['Грузовой лифт'] + apartments['Пассажирский лифт']).apply(lambda x: 1 if x > 0 else 0)

In [14]:
apartments['Количество комнат'].unique()

array(['3', '2', '1', 'студия', '9', '4'], dtype=object)

In [15]:
apartments['Количество комнат'].replace('студия', 1, inplace=True)

# Cleaning categorical features

In [16]:
apartments['Улица'] = apartments['Адрес'].str.replace('Курская область, Курск', '').str.strip(',').str.split(',').str[-2].str.strip()
apartments[apartments['Улица'].isnull()]

,Адрес,Район,Цена,Количество комнат,Общая площадь,Площадь кухни,Этаж,Балкон или лоджия,Тип комнат,Санузел,...,Можно с детьми,Можно с животными,Можно курить,Тип дома,Этажей в доме,Пассажирский лифт,Грузовой лифт,Парковка,Лифт,Улица
33,"Курская область, Курск, ул. Щепкина",р-н Центральный,6500.0,1,16.0,2.0,5,NaN,NaN,раздельный,...,да,да,да,кирпичный,9,1,0,за шлагбаумом во дворе,1,NaN
88,"Курская область, Курск, пр-т Анатолия Дериглазова",р-н Центральный,9000.0,1,38.0,10.0,17,NaN,NaN,NaN,...,да,нет,нет,панельный,17,0,0,NaN,0,NaN
199,"Курская область, Курск, пр-т Вячеслава Клыкова",р-н Центральный,13000.0,2,56.1,10.9,3,балкон,изолированные,раздельный,...,нет,нет,нет,панельный,17,1,1,открытая во дворе,1,NaN


In [17]:
apartments['Улица'].fillna(apartments['Адрес'].str.split(',').str[-1].str.strip(), inplace=True)
apartments['Улица'].isnull().sum()

0

In [27]:
apartments['Улица'].unique()

array(['ул. Карла Либкнехта', 'Красная пл.', 'пр-т Вячеслава Клыкова',
       'пр-т Победы', 'Студенческая ул.', 'ул. Горького', 'пр-т Хрущёва',
       'ул. Блинова', 'пр-т Надежды Плевицкой', 'Союзная ул.',
       'Почтовая ул.', 'ул. 50 лет Октября', 'ул. Дзержинского',
       'ул. Карла Маркса', 'ул. Менделеева', 'Звёздная ул.',
       'ул. Бойцов 9-й Дивизии', 'пр-т Дружбы', 'ул. Черняховского',
       'Школьная ул.', 'ул. Косухина', 'пр-т Ленинского Комсомола',
       '1-я Ламоновская ул.', 'пр-т Анатолия Дериглазова', 'ул. Щепкина',
       'ул. Крюкова', 'ул. Радищева', 'ул. Ленина', 'ул. Володарского',
       'ул. Ватутина', 'ул. Димитрова', 'Магистральный пр.',
       'ул. Домостроителей', 'ул. Павлуновского', 'Хуторская ул.',
       'ул. Дейнеки', 'ул. Льва Толстого', 'ул. Челюскинцев',
       'Мирная ул.', 'ул. Гоголя', 'Московский пр.', 'ул. Чапаева',
       'Советская ул.', 'ул. Пионеров', '2-я ул. Новосёловка',
       'ул. Гайдара', 'Институтская ул.', 'ул. Чернышевского',

In [18]:
apartments.drop('Адрес', axis=1, inplace=True)

In [19]:
def get_columns_counts_info(df, columns):
    for column in columns:
        print(column)
        print(df[column].value_counts())
        print(f'NaN: {df[column].isnull().sum()}\n')

In [20]:
#get_columns_counts_info(apartments,['Балкон или лоджия', 'Парковка', 'Тип комнат', 'Санузел', 'Ремонт', 'Мебель', 'Техника', 'Интернет и ТВ'])

def contains_substring_to_binary(value, substring):
    if value is None:
        return 0
    if substring in str(value):
        return 1
    return 0

apartments['Балкон'] = apartments['Балкон или лоджия'].apply(lambda x: contains_substring_to_binary(x, 'балкон'))
apartments['Лоджия'] = apartments['Балкон или лоджия'].apply(lambda x: contains_substring_to_binary(x, 'лоджия'))
#apartments.drop('Балкон или лоджия', axis=1, inplace=True)

In [21]:
cols = 'Можно с детьми', 'Можно с животными', 'Можно курить'
get_columns_counts_info(apartments, cols)

Можно с детьми
да                              147
нет                              66
Можно с детьмиМожно с детьми      1
5Дополнительно                    1
Name: Можно с детьми, dtype: int64
NaN: 0

Можно с животными
нет    185
да      30
Name: Можно с животными, dtype: int64
NaN: 0

Можно курить
нет    161
да      20
Name: Можно курить, dtype: int64
NaN: 34



In [22]:
def contains_substring_to_binary(value, substring):
    if value is None:
        return 0
    if substring in str(value):
        return 1
    return 0

for col in cols:
    apartments[col] = apartments[col].apply(lambda x: contains_substring_to_binary(x, 'да'))

In [23]:
apartments.head()

,Район,Цена,Количество комнат,Общая площадь,Площадь кухни,Этаж,Балкон или лоджия,Тип комнат,Санузел,Ремонт,...,Можно с детьми,Можно с животными,Можно курить,Тип дома,Этажей в доме,Пассажирский лифт,Грузовой лифт,Парковка,Лифт,Улица
0,р-н Центральный,35000.0,3,116.0,20.0,5,"балкон, лоджия",изолированные,раздельный,евро,...,1,0,0,кирпичный,9,1,0,за шлагбаумом во дворе,1,ул. Карла Либкнехта
1,р-н Центральный,27000.0,3,80.0,15.0,2,балкон,изолированные,совмещенный,евро,...,1,0,0,кирпичный,5,0,0,за шлагбаумом во дворе,0,Красная пл.
2,р-н Центральный,12000.0,2,58.0,12.0,1,балкон,изолированные,раздельный,косметический,...,1,0,0,панельный,17,1,1,открытая во дворе,1,пр-т Вячеслава Клыкова
3,р-н Центральный,13000.0,2,75.0,15.0,5,лоджия,изолированные,NaN,косметический,...,1,0,0,монолитный,10,1,0,открытая во дворе,1,пр-т Победы
4,р-н Центральный,15000.0,3,85.0,13.0,9,лоджия,"изолированные, смежные",раздельный,евро,...,1,0,0,панельный,10,0,0,открытая во дворе,0,Студенческая ул.


In [24]:
house_types = apartments.groupby('Тип дома')['Тип дома'].agg('count').sort_values(ascending=False)
house_types

Тип дома
панельный              106
кирпичный               74
монолитный              24
монолитно-кирпичный      9
блочный                  1
деревянный               1
Name: Тип дома, dtype: int64

In [25]:
#apartments['Тип дома'] = apartments['Тип дома'].apply(lambda x: 'другой' if x in house_types[house_types <= 10] else x)

In [26]:
apartments.to_csv("Data/kursk_apartments.csv")